# 使用 Nsight Compute 优化在线 5G 机器学习

## 4.1 Nsight Compute CUDA 内核分析器

我们将 CUDA 内核 `kernel_apsm_detect` 确定为我们的优化目标。要详细了解其性能，我们可以使用 Nsight Compute 对其进行分析。让我们先简单介绍一下该工具：

Nsight Compute 是一个 `交互式 CUDA 内核分析器`，具有：
* 针对各种性能方面的目标指标部分
* 可自定义的数据收集和呈现（表格、图表等）
* UI 和命令行
* 基于 Python 的引导分析（或后处理）规则
* 支持跨机器和平台的远程分析

<img src="images/ncu_intro_01.png" width="900">

详细的 `内存工作负载分析` 图表和表格有助于了解不同硬件单元之间的瓶颈以及它们的利用效率。该工具支持将大多数图表和表格中的数据与一个或多个基线进行比较，以查看对代码的任何优化的影响。支持跨内核、报告和 GPU 架构进行比较。

<img src="images/ncu_intro_02.png" width="900">

`Source` 页面提供高级 CUDA C/C++ 源代码、PTX 和 SASS（汇编）之间的关联。每个指令都有多个指标，用于对源代码进行详细的“逐行分析”。指标热图有助于快速找到特定指标的热点。

<img src="images/ncu_intro_03.png" width="900">

## 4.2 交互式分析 kernel_apsm_detect

在开始研究内核性能之前，请先查看 [apsm_versions.h](apsm/cpp/lib/apsm/apsm_versions.h)：您会发现这里有多个可用的内核实现，可通过设置 `APSM_DETECT_VERSION` 定义进行选择。我们已经从 `Cooperative Groups` (CG) 实现 `APSM_DETECT_CG` 开始，但还有一个没有 CG 的 `ORIGINAL` 版本，如果您有兴趣，可以稍后进行比较。

我们将在远程桌面环境中使用 Nsight Compute UI。执行以下单元格以生成远程桌面的 URL，您应将其复制并粘贴到新的浏览器选项卡中。noVNC 密码是 `nvidia` 。然后继续按照演示者或以下说明进行操作。

In [ ]:
%%js
var url = window.location.host + '/nsight/';
element.append(url)

单击下方 `...` 以查看注释。

切换到 Ubuntu 实例（密码为 `nvidia`），使用搜索/放大镜图标打开“查找应用程序”工具，然后搜索 `compute` 以选择 Nsight Compute。在该工具中，通过点击“加载项目”并选择`/root/Desktop/reports/ncu/apsm.ncu-proj`项目文件来打开准备好的项目。这将打开连接对话框，该对话框现在已预先填充了应用程序详细信息。

<img src="images/ncu_connect.png" width="700">

启动后，Nsight Compute 连接到目标应用程序并在第一个 CUDA API 调用中将其暂停，该调用在 API Stream 工具窗口中可见。由于我们想要分析 `kernel_apsm_detect` 内核，请在 `Next Trigger` 编辑中输入该名称，然后选择绿色的 `Run to Next Kernel` 按钮。这允许应用程序继续运行，直到启动该内核之前。

在开始分析之前，请启用 `Sections/Rules Info` 工具窗口中的 `full` 部分设置，以便让 Nsight Compute 收集全套精选指标。由于我们只有一个内核需要分析，因此我们不太担心多次重放内核时的开销。之后，单击 `Profile Kernel` 并等待报告创建。最后，我们可以“终止”目标应用程序。

<img src="images/ncu_full_set.png" width="700">

现在我们可以开始分析创建的分析器报告。在“详细信息”页面上，从上到下检查各个部分，并注意工具生成的“建议”。

第一部分显示内核对 SM 计算单元的利用率较低（`Compute (SM) Throughput`），并且内存单元的吞吐量也相对较低（`Memory Throughput`）。该工具表明内核受延迟限制，我们应该继续查看“调度程序”和“Warp 状态统计”部分。但是，您也可以随意检查“计算和内存工作负载分析”部分中显示的信息。

<img src="images/ncu_report01_01.png">

在“调度程序统计”部分，我们可以看到每个调度程序的理论 Warp 数量仅为硬件能力的一半。因此，“活动 Warp”也低于 GPU 最大值。虽然这本身并不成问题，但我们可以看到调度程序“发出”的每个周期的 Warp 数量远低于一个，导致调度工作之间存在多个周期的延迟。

<img src="images/ncu_report01_02.png">

该工具建议减少 Warp 停顿（我们可以在 `Warp State Statistics` 部分进行调查），或者增加活动 Warp 的数量（从而也符合条件/已发出）。虽然我们可以从任何一种方式开始，但我们的“理论 Warp”只有 GPU 硬件最大值的一半，这一事实表明我们甚至没有“占用”全部可用硬件。首先让内核拥有所有可用的 SM 计算单元，然后优化每个单元的使用情况，这可能是一个好策略。
​​
要继续这条路，我们可以使用 `Occupancy` 部分进行进一步输入。点击链接进入 Occupancy 部分。

<img src="images/ncu_report01_03.png">

我们可以看到，我们的理论占用率为 50%，实际占用率约为 49%。每个调度程序的 Warp 数和每个 SM 的活动 Warp 数之间的密切关系还体现在，在本节中，我们可以看到“每个 SM 的活动 Warp 数”与理论值和实际值之间的相同关系。因此，将我们的内核更改为具有 100% 的理论占用率也可能导致每个调度程序的理论和实际 Warp 数同样得到改善。

为了更好地理解启动配置选项（如块大小）对理论占用率的影响，我们可以使用 `占用率计算器（Occupancy Calculator）` 活动。要使用当前内核参数预填充计算器，只需单击“占用率”部分中的标记按钮即可。

<img src="images/ncu_report01_04.png">

在占用率计算器中，您可以查找 GPU 限制，查看当前占用率限制器并尝试不同的值以查看理论占用率如何变化。占用率目前受到（小）块大小的限制，每个块只有一个 Warp（32 个线程）。如果我们可以将块大小加倍至 64 个线程（或 2 个 warp），那么内核将占据 GPU 的所有多处理器。

分析之后，我们可以继续在[步骤 05](05_spb.ipynb) 中优化和重新评估内核。